In [1]:
import sys
sys.path.append("/home/wentaos/Optima/local_influence/")

In [2]:
import yaml
with open("/home/wentaos/Optima/local_influence/recipe/hotpotqa/iteration_0_dpo.yaml", "r") as config_f:
    config = yaml.safe_load(config_f)

In [3]:
from accelerate import Accelerator
import numpy as np
import random
import torch

# set_seed(42)
torch.manual_seed(42)
random.seed(42)
np.random.seed(42)
torch.backends.cudnn.deterministic = True



In [4]:
from pathlib import Path
import argparse
import random
import os
from datetime import datetime
from typing import Dict, Tuple

import numpy as np
import torch
import yaml
from accelerate import Accelerator
from datasets import Dataset, load_from_disk
from torch.optim import AdamW
from torch.utils.data import DataLoader
from tqdm import tqdm
from transformers import AutoModelForCausalLM, AutoTokenizer, set_seed
import transformers
# from utils import PartialMaskTokenizer, reformat_to_chat

from transformers import LlamaForCausalLM, LlamaTokenizer
from peft import PeftModel, PeftConfig
from my_utils import (get_tokenizer, process_dpo_format_to_dataset, 
        apply_chat_template, decontaminate_humaneval, dpo_generate_and_tokenize_prompt, 
        SFT_generate_and_tokenize_prompt, PreferenceCollator, DataArguments, get_datasets)


import os

/home/wentaos/miniconda3/envs/trak/lib/python3.10/site-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
/home/wentaos/miniconda3/envs/trak/lib/python3.10/site-packages/torchvision/transforms/v2/__init__.py:54: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit 

In [5]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]='0'

os.environ["HF_HOME"]="~/.cache/huggingface"

accelerator = Accelerator(mixed_precision="bf16")
accelerator.print(
    f"datetime: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}", flush=True
)


datetime: 2024-11-13 18:18:38


In [20]:




peft_config = PeftConfig.from_pretrained(config["lora_path"])
# peft_config.base_model_name_or_path = config["model_name_or_path"]



raw_model = LlamaForCausalLM.from_pretrained(config["model_name_or_path"], 
                                            torch_dtype=torch.float16,
                                            device_map="cuda",
                                            cache_dir=os.environ["HF_HOME"],
                                            )
raw_model = PeftModel.from_pretrained(raw_model, config["lora_path"], torch_dtype=torch.float16, adapter_name="first adapter")
for n, p in raw_model.named_parameters():
    if 'lora' in n:
        print(n)
        p.requires_grad = True
# raw_model = AutoModelForCausalLM.from_pretrained(
#     config["model_name_or_path"],
#     torch_dtype=torch.bfloat16,
#     device_map="cpu",
#     cache_dir=os.environ["HF_HOME"],
# )
model = accelerator.prepare(raw_model)



Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

base_model.model.model.layers.0.self_attn.q_proj.lora_A.first adapter.weight
base_model.model.model.layers.0.self_attn.q_proj.lora_B.first adapter.weight
base_model.model.model.layers.0.self_attn.k_proj.lora_A.first adapter.weight
base_model.model.model.layers.0.self_attn.k_proj.lora_B.first adapter.weight
base_model.model.model.layers.0.self_attn.v_proj.lora_A.first adapter.weight
base_model.model.model.layers.0.self_attn.v_proj.lora_B.first adapter.weight
base_model.model.model.layers.0.self_attn.o_proj.lora_A.first adapter.weight
base_model.model.model.layers.0.self_attn.o_proj.lora_B.first adapter.weight
base_model.model.model.layers.0.mlp.gate_proj.lora_A.first adapter.weight
base_model.model.model.layers.0.mlp.gate_proj.lora_B.first adapter.weight
base_model.model.model.layers.0.mlp.up_proj.lora_A.first adapter.weight
base_model.model.model.layers.0.mlp.up_proj.lora_B.first adapter.weight
base_model.model.model.layers.0.mlp.down_proj.lora_A.first adapter.weight
base_model.model.m

In [21]:
from collect import get_dpo_dataloader
tokenizer = get_tokenizer(config["tokenizer_path"], truncation_side="left") 

train_args, eval_args, tokenizer_path = (
    config["train_args"],
    config["eval_args"],
    config["tokenizer_path"],
)


In [8]:
train_dataloader = get_dpo_dataloader(accelerator, train_args, "train", tokenizer, batch_size=1)

Formatting comparisons with prompt template:   0%|          | 0/14679 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1521 > 1024). Running this sequence through the model will result in indexing errors


Formatting comparisons with prompt template:   0%|          | 0/1632 [00:00<?, ? examples/s]

Decontaminating HumanEval samples:   0%|          | 0/14679 [00:00<?, ? examples/s]

Decontaminating HumanEval samples:   0%|          | 0/1632 [00:00<?, ? examples/s]

Map:   0%|          | 0/14679 [00:00<?, ? examples/s]

Map:   0%|          | 0/1632 [00:00<?, ? examples/s]

original train dataset:DatasetDict({
    train: Dataset({
        features: ['prompt', 'chosen', 'rejected'],
        num_rows: 14679
    })
    test: Dataset({
        features: ['prompt', 'chosen', 'rejected'],
        num_rows: 1632
    })
})



In [9]:
eval_dataloader = get_dpo_dataloader(accelerator, eval_args, "test", tokenizer, batch_size=1)

Formatting comparisons with prompt template:   0%|          | 0/14025 [00:00<?, ? examples/s]

Formatting comparisons with prompt template:   0%|          | 0/1559 [00:00<?, ? examples/s]

Decontaminating HumanEval samples:   0%|          | 0/14025 [00:00<?, ? examples/s]

Decontaminating HumanEval samples:   0%|          | 0/1559 [00:00<?, ? examples/s]

Map:   0%|          | 0/14025 [00:00<?, ? examples/s]

Map:   0%|          | 0/1559 [00:00<?, ? examples/s]

original train dataset:DatasetDict({
    train: Dataset({
        features: ['prompt', 'chosen', 'rejected'],
        num_rows: 14025
    })
    test: Dataset({
        features: ['prompt', 'chosen', 'rejected'],
        num_rows: 1559
    })
})



In [10]:
# from collect import get_sft_dataloader
# eval_dataloader = get_sft_dataloader(accelerator, eval_args, "test", tokenizer)

optimizer = AdamW(params=model.parameters())

optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    optimizer, train_dataloader, eval_dataloader
)


In [11]:
from my_utils import pad_to_length
import torch.nn.functional as F
from torch.func import functional_call, vmap
max_completion_length = 1024
if "llama-3":
    padding_value = 128002
    
def compute_dpo_loss(model, prompt_input_ids, prompt_attention_mask, chosen_input_ids, chosen_attention_mask, rejected_input_ids, rejected_attention_mask):

    rpo_alpha = 1.0
    beta = 0.1
    max_length = 1024

    
    
    num_examples = prompt_input_ids.shape[0]
   
    
    # For the prompt, the input_ids are the same for both the chosen and rejected responses
    prompt_input_ids = torch.cat([prompt_input_ids, prompt_input_ids], dim=0)
    prompt_attention_mask = torch.cat(
        [prompt_attention_mask, prompt_attention_mask], dim=0
    )

    

    # Concatenate the chosen and rejected completions
    max_completion_length = max(chosen_input_ids.shape[1], rejected_input_ids.shape[1])
    completion_input_ids = torch.cat(
        (
            pad_to_length(chosen_input_ids, max_completion_length, pad_value=padding_value),
            pad_to_length(rejected_input_ids, max_completion_length, pad_value=padding_value),
        ),
    )
    completion_attention_mask = torch.cat(
        (
            pad_to_length(chosen_attention_mask, max_completion_length, pad_value=0),
            pad_to_length(rejected_attention_mask, max_completion_length, pad_value=0),
        ),
    )

    
    input_ids = torch.cat((prompt_input_ids, completion_input_ids), dim=1)
    attention_mask = torch.cat((prompt_attention_mask, completion_attention_mask), dim=1)
    # Mask the prompt but not the completion for the loss
    loss_mask = torch.cat(
        (torch.zeros_like(prompt_attention_mask), completion_attention_mask),
        dim=1,
    )

    if max_length is not None:
        input_ids = input_ids[:, : max_length]
        attention_mask = attention_mask[:, : max_length]
        loss_mask = loss_mask[:, : max_length]
    # # Flush left to reduce the memory usage
    # # [[0, 0, x, x, x, x],  ->  [[x, x, x, x],
    # #  [0, x, x, x, 0, 0]]       [x, x, x, 0]]
    # for i in range(attention_mask.size(0)):
    #     first_one_idx = torch.nonzero(attention_mask[i])[0].item()
    #     input_ids[i] = torch.roll(input_ids[i], shifts=-first_one_idx)
    #     attention_mask[i] = torch.roll(attention_mask[i], shifts=-first_one_idx)
    #     loss_mask[i] = torch.roll(loss_mask[i], shifts=-first_one_idx)

    # # Get the first column idx that is all zeros and remove every column after that
    # empty_cols = torch.sum(attention_mask, dim=0) == 0
    # first_empty_col = torch.nonzero(empty_cols)[0].item() if empty_cols.any() else attention_mask.size(1)
    # input_ids = input_ids[:, :first_empty_col]
    # attention_mask = attention_mask[:, :first_empty_col]
    # loss_mask = loss_mask[:, :first_empty_col]

    labels = input_ids[:, 1:].clone()
    loss_mask = loss_mask[:, 1:].bool()
    # labels[~loss_mask] = 0
    labels = torch.where(loss_mask, labels, torch.tensor(0, dtype=labels.dtype, device=labels.device))
    
    outputs = model(input_ids=input_ids, 
                                  kwargs={'attention_mask': attention_mask, 
                                          # 'labels': labels
                                         })
    
    
    logits = outputs.logits[:, :-1, :]
    per_token_logps = torch.gather(logits.log_softmax(-1), dim=2, index=labels.unsqueeze(2)).squeeze(2)
    # per_token_logps[~loss_mask] = 0
    per_token_logps = torch.where(loss_mask, per_token_logps, torch.tensor(0, dtype=per_token_logps.dtype, device=per_token_logps.device))
    all_logps = per_token_logps.sum(-1)
    chosen_logps = all_logps[:num_examples]
    rejected_logps = all_logps[num_examples:]

    
        
    if rpo_alpha is not None:
        chosen_logits = logits[:num_examples]
        chosen_labels = labels[:num_examples]

        # Compute the log probabilities of the labels
        nll_loss = F.cross_entropy(
            torch.flatten(chosen_logits, end_dim=1), torch.flatten(chosen_labels, end_dim=1), ignore_index=0
        )

    with model.disable_adapter():
        
        ref_outputs = model(input_ids=input_ids, 
                                  kwargs={'attention_mask': attention_mask, 
                                          # 'labels': labels
                                         })
        ref_logits = ref_outputs.logits[:, :-1, :]
        ref_per_token_logps = torch.gather(ref_logits.log_softmax(-1), dim=2, index=labels.unsqueeze(2)).squeeze(2)
        # ref_per_token_logps[~loss_mask] = 0
        ref_per_token_logps = torch.where(loss_mask, ref_per_token_logps, torch.tensor(0, dtype=ref_per_token_logps.dtype, device=ref_per_token_logps.device))
        ref_all_logps = ref_per_token_logps.sum(-1)
        ref_chosen_logps = ref_all_logps[:num_examples]
        ref_rejected_logps = ref_all_logps[num_examples:]


    
    pi_logratios = chosen_logps - rejected_logps
    ref_logratios = ref_chosen_logps - ref_rejected_logps
    differences = pi_logratios - ref_logratios
    losses = -F.logsigmoid(beta * differences)
    
    loss = losses + rpo_alpha * nll_loss
    
    # print(loss)
    return loss.squeeze(0) # must be a scaler

In [12]:
torch.cuda.empty_cache()
!nvidia-smi

Wed Nov 13 18:21:06 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          On  |   00000000:87:00.0 Off |                    0 |
| N/A   33C    P0             68W /  380W |   16159MiB /  40960MiB |      3%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [18]:
def one_step_train(
    model: AutoModelForCausalLM,
    optimizer: AdamW,
    accelerator: Accelerator,
    data_batch: Dict[str, torch.Tensor],
    eval_dataloader: DataLoader,
    step: int,
    avg_loss_before: float,
) -> float:
    device = "cpu"
    model.to(device)
    model.train()
    optimizer.zero_grad()
    data_batch = {k: v.to(device) for k, v in data_batch.items()}
    loss = compute_dpo_loss(model, data_batch['prompt_input_ids'], data_batch['prompt_attention_mask'], 
                                data_batch['chosen_input_ids'], data_batch['chosen_attention_mask'],
                                data_batch['rejected_input_ids'], data_batch['rejected_attention_mask'])
    
    loss = loss / accelerator.num_processes
    # if accelerator.is_local_main_process:
    #     print(f"{accelerator.device} loss: {loss:.10f}")
    # accelerator.print("train loss: ", loss.item())
    # print(f"train loss {loss.item()} on device {accelerator.device}")
    accelerator.backward(loss)
    optimizer.step()
    optimizer.zero_grad()

    # eval
    model.eval()
    total_loss = 0
    for index, batch in tqdm(
        enumerate(eval_dataloader),
        total=len(eval_dataloader),
        desc="Evaluating",
        disable=not accelerator.is_local_main_process,
    ):
        batch = {k: v.to(device) for k, v in batch.items()}
    
        with torch.no_grad():
            loss = compute_dpo_loss(model, batch['prompt_input_ids'], batch['prompt_attention_mask'], 
                                                        batch['chosen_input_ids'], batch['chosen_attention_mask'],
                                                        batch['rejected_input_ids'], batch['rejected_attention_mask'])
            
        loss_withoutnan = torch.nanmean(accelerator.gather(loss))
        total_loss += loss_withoutnan.item()
    avg_loss_after = total_loss / len(eval_dataloader)
    accelerator.print(f"eval loss after one step train: {avg_loss_after}")
    accelerator.print(f"eval loss diff: {avg_loss_after - avg_loss_before}")
    return avg_loss_after

In [ ]:

# obtain baseline reference loss
model.eval()
total_loss = 0
device = "cuda"
    


# for index, batch in tqdm(
#     enumerate(eval_dataloader),
#     total=len(eval_dataloader),
#     desc="Evaluating",
#     disable=not accelerator.is_local_main_process,
# ):
#     batch = {k: v.to(device) for k, v in batch.items()}
    
#     with torch.no_grad():
#         loss = compute_dpo_loss(model, batch['prompt_input_ids'], batch['prompt_attention_mask'], 
#                                                      batch['chosen_input_ids'], batch['chosen_attention_mask'],
#                                                      batch['rejected_input_ids'], batch['rejected_attention_mask'])
#         # print(loss)
#     # loss = outputs.loss
#     loss_withoutnan = torch.nanmean(accelerator.gather(loss))
#     loss = accelerator.gather(loss).mean()
#     # print(loss)
#     # print(loss_withoutnan)
#     total_loss += loss_withoutnan.item()
# avg_loss_before = total_loss / len(eval_dataloader)
# accelerator.print(f"baseline reference loss: {avg_loss_before}")
# # Write the baseline reference loss to a file
# baseline_ref_loss_path = os.path.join(
#     Path(config["train_args"]["data_path"]).parent, "baseline_ref_loss.txt"
# )
# print(f"baseline reference loss path: {baseline_ref_loss_path}")
# with open(baseline_ref_loss_path, "w") as f:
#     f.write(f"{avg_loss_before}")
avg_loss_before = 0

score_list = []
for step, batch in tqdm(
    enumerate(train_dataloader),
    total=len(train_dataloader),
    desc="Training",
    disable=not accelerator.is_local_main_process,
):
    accelerator.print(f"..........train batch: {step}...........")
    t1 = datetime.now()
    loss = one_step_train(
        model, optimizer, accelerator, batch, eval_dataloader, step, avg_loss_before
    )
    # accelerator.print(
    #     f"Epoch {epoch}, Step {step}, Loss on {accelerator.device}: {loss}"
    # )
    score_list.append(loss)
    # accelerator.load_state(config["warm_up_path"])
    t2 = datetime.now()
    accelerator.print(f"time taken: {(t2 - t1).total_seconds()}")
# print(score_list)
# raw_train_dataset = raw_train_dataset.add_column("scores", score_list)
# raw_train_dataset.save_to_disk(config["scores_dataset_path"])
accelerator.end_training()

Training:   0%|                                                              | 0/1632 [00:00<?, ?it/s]

..........train batch: 0...........
